In [1]:
import os
data_paths = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        data_paths.append(os.path.join(dirname, filename))


/kaggle/input/my-data/customers_zip.csv
/kaggle/input/my-data/delivery_zip.csv
/kaggle/input/my-data/sports.csv
/kaggle/input/my-data/orders.csv
/kaggle/input/my-data/customer_orders.csv


In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 16.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=29ffff4b82b2ab531277597999f9987921f31f85234e4bf6778f55c70898af69
  Stored in directory: /root/.cache/pip/wheels/5a/54/9b/a89cac960efb57c4c35d41cc7c9f7b80daa21108bc376339b7
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pyspark 
from pyspark.sql.types import * 
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import count as sql_count


In [5]:
spark = SparkSession.builder.appName('my_spark').getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/28 13:00:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Loading data

In [6]:
# creating dataframes

for file_path in data_paths:
    file_name = os.path.basename(file_path).split('.')[0]
    globals()[file_name] = spark.read.csv(file_path, header=True, inferSchema=True)



In [7]:
df_names = [os.path.splitext(os.path.basename(path))[0] for path in data_paths]
df_names

['customers_zip', 'delivery_zip', 'sports', 'orders', 'customer_orders']

# Data frames:

# customers_zip

In [8]:
customers_zip = customers_zip.drop('_c0')
customers_zip.show(5)
customers_zip.printSchema()
customers_zip.count() 

+-----------+--------+
|customer_id|zip_cust|
+-----------+--------+
|   77461851|   39435|
|   37934116|   92635|
|   39770481|   61090|
|   50298765|   40516|
|   23383753|   85367|
+-----------+--------+
only showing top 5 rows

root
 |-- customer_id: string (nullable = true)
 |-- zip_cust: string (nullable = true)



1064035

In [9]:
# casting, zmiana typu danych string na poprawny typ, integer. 

customers_zip = customers_zip.withColumn('customer_id', col('customer_id').cast(IntegerType()))
customers_zip = customers_zip.withColumn('zip_cust', col('zip_cust').cast(IntegerType()))

customers_zip.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- zip_cust: integer (nullable = true)



In [10]:
# usuniecie wierszy z wartosciami NULL 
customers_zip = customers_zip.dropna()
customers_zip.count()

1064034

# delivery_zip

In [11]:
delivery_zip = delivery_zip.drop('_c0')
delivery_zip.show(5)
delivery_zip.printSchema()
delivery_zip.count() 

+-----------+-------+
|customer_id|zip_del|
+-----------+-------+
|   77461851|  55685|
|   37934116|  36236|
|   39770481|     NA|
|   50298765|  78823|
|   23383753|  75757|
+-----------+-------+
only showing top 5 rows

root
 |-- customer_id: string (nullable = true)
 |-- zip_del: string (nullable = true)



1701452

In [12]:
# casting, musimy zmienic typ danych ze string na integer, sprawdzamy ponownie za pomoca printSchema czy dane sa poprawne 

delivery_zip = delivery_zip.withColumn('customer_id', col('customer_id').cast(IntegerType()))
delivery_zip = delivery_zip.withColumn('zip_del', col('zip_del').cast(IntegerType()))

delivery_zip.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- zip_del: integer (nullable = true)



In [13]:
# usuniecie wierszy z wartosciami NULL 
delivery_zip = delivery_zip.dropna()
delivery_zip.count()

1651092

# sports

In [14]:
sports = sports.drop('_c0')
sports.show(5)
sports.printSchema()
sports.count()

+-----------+----------+
|customer_id|     sport|
+-----------+----------+
|   77461851|  p3ywanie|
|   37934116| myolistwo|
|   39770481|3ucznictwo|
|   50298765|nurkowanie|
|   23383753|3ucznictwo|
+-----------+----------+
only showing top 5 rows

root
 |-- customer_id: string (nullable = true)
 |-- sport: string (nullable = true)



1189747

In [15]:
sports = sports.withColumn('customer_id', col('customer_id').cast(IntegerType()))
sports.printSchema() 

root
 |-- customer_id: integer (nullable = true)
 |-- sport: string (nullable = true)



In [16]:
# usuniecie wierszy z wartosciami NULL 
sports = sports.dropna()
sports.count()

1189746

# orders

In [17]:
orders = orders.drop('_c0')
orders.show(5)
orders.printSchema()
orders.count()

+--------+------+
|order_id| value|
+--------+------+
|82168832|240.29|
|38300086|730.12|
|70629629|145.04|
|81244871|472.24|
|88005542|354.54|
+--------+------+
only showing top 5 rows

root
 |-- order_id: string (nullable = true)
 |-- value: string (nullable = true)



4003982

In [18]:
orders = orders.withColumn('order_id', col('order_id').cast(IntegerType()))
orders = orders.withColumn('value', col('value').cast(IntegerType()))

orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- value: integer (nullable = true)



In [19]:
# usuniecie wierszy z wartosciami NULL 
orders = orders.dropna()
orders.count()

4003977

# customer_orders

In [20]:
customer_orders = customer_orders.drop('_c0')
customer_orders.show(5)
customer_orders.printSchema()
customer_orders.count()

+-----------+--------+
|customer_id|order_id|
+-----------+--------+
|   77461851|82168832|
|   37934116|38300086|
|   39770481|70629629|
|   50298765|81244871|
|   23383753|88005542|
+-----------+--------+
only showing top 5 rows

root
 |-- customer_id: string (nullable = true)
 |-- order_id: string (nullable = true)



4003964

In [21]:
customer_orders = customer_orders.withColumn('customer_id', col('customer_id').cast(IntegerType()))
customer_orders = customer_orders.withColumn('order_id', col('order_id').cast(IntegerType()))

orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- value: integer (nullable = true)



In [22]:
# usuniecie wierszy z wartosciami NULL 
customer_orders = customer_orders.dropna()
customer_orders.count()

4003963

# 1. Jaki odsetek klientów odbiera swoje zamówienia pod innym kodem pocztowym niż mieszka?


In [23]:
# Step 1, polaczenie tabeli 'customers_zip' i 'delivery_zip'

joined_df = customers_zip.join(delivery_zip, ['customer_id'], 'left')


In [24]:
# sprawdzenie liczby klientow ktorzy orzymuja swoje zamowienia pod innym kodem pocztowym
diff_zip_count = joined_df.filter(joined_df.zip_cust != joined_df.zip_del)
diff_zip_count.count()

1013072

In [25]:
# suma wszytskich klientow
total_count = joined_df.select('customer_id').distinct().show()


+-----------+
|customer_id|
+-----------+
|     570578|
|   36025674|
|   81174526|
|   43802989|
|   73127079|
|   29347835|
|     607933|
|   55250873|
|    7697744|
|   60001774|
|   86712831|
|   24136461|
|   46584318|
|   60878122|
|   19750152|
|   55151857|
|   16422510|
|   85269095|
|   78237449|
|   15057151|
+-----------+
only showing top 20 rows



In [26]:
# obliczenie procentu klientow ktorzy otrzymuja zamowienia pod innym kodem pocztowym
percentage_diff_zip = (1013072/1060705) * 100
print(percentage_diff_zip)

95.5093074888871


 **95%** procent klientow odbiera swoje zamowienia pod innym kodem pocztowym niz mieszka.

# 2. Jaki sport jest najbardziej i najmniej popularny wśród klientów?

In [27]:
# polaczenie 'customers_zip' i 'sports' 
customers_orders = customers_zip.join(sports, 'customer_id', 'left')

In [28]:
# pogrupowanie na rodzaje sportow malejaco, od najbardzienpopularnego
grouped_customers_orders = customers_orders.groupBy('sport').agg(count('customer_id').alias('customer_count')).orderBy(desc('customer_count'))
grouped_customers_orders.show()

+-----------+--------------+
|      sport|customer_count|
+-----------+--------------+
|   p3ywanie|        275734|
| 3ucznictwo|        223552|
| nurkowanie|        212356|
|  myolistwo|        181253|
|jeYdziectwo|         63859|
|   bieganie|         53269|
|     rowery|         43153|
|     karate|         21695|
|aquafitness|         21566|
|       judo|         21476|
|  snowboard|         21472|
|  chodzenie|         21423|
|      hokej|         21409|
|  myslistwo|         10858|
+-----------+--------------+



Na podstawie danych mozemy stwierdzic iz **plywanie** jest **najpopularniejszym** sportem wsrod klientow, zas **myslistwo** jest **najmniej** **popularne**

# 3. Ilu klientów uprawia więcej niż 2 sporty?

In [29]:
# polaczenie 'customers_zip' i 'sports'
zip_sports = customers_zip.join(sports, 'customer_id', 'left')
zip_sports.show(5)

+-----------+--------+----------+
|customer_id|zip_cust|     sport|
+-----------+--------+----------+
|    2789905|   93737|  p3ywanie|
|    7280623|   22931|3ucznictwo|
|   37934116|   92635| myolistwo|
|   50298765|   40516|nurkowanie|
|   70237995|   38540| myolistwo|
+-----------+--------+----------+
only showing top 5 rows



In [30]:

grouped_zip_sports = zip_sports.groupBy('customer_id').agg(sql_count('sport').alias('sport_count'))


In [31]:
# sprawdzenie ilu klientow uprawia wiecej niz 2 sporty
num_customers = grouped_zip_sports.filter(grouped_zip_sports.sport_count > 2)
num_customers.count()

5040

Sposrod wszystkich klientow, **5040** uprawia wiecej niz 2 dyscypliny

# 4. Jaka jest średnia wartość zamówienia?

In [32]:
avg_order_value = orders.select(mean('value'))
avg_order_value.show()

+------------------+
|        avg(value)|
+------------------+
|25388.560869605393|
+------------------+



In [33]:
orders.select('value').summary("count", "min", "25%", "50%", "75%", "max", "stddev", "mean").show()


+-------+------------------+
|summary|             value|
+-------+------------------+
|  count|           4003977|
|    min|              -800|
|    25%|               203|
|    50%|               405|
|    75%|               606|
|    max|           7999900|
| stddev| 404609.2541032316|
|   mean|25388.560869605393|
+-------+------------------+



Srednia wartosc zamowienia wynosi **25388**, jednak w przypadku tej tabeli, odchylenie standardowe(**404609**) jest bardzo duże w porównaniu ze średnią, co wskazuje na duże zróżnicowanie w danych. W tym przypadku mediana (**405**) byłaby lepszym wyborem, ponieważ bardziej reprezentuje "typowe" wartości w zestawie danych.

# 5. Jaka jest średnia wartość zamówienia klientów uprawiających jeździectwo?

In [34]:

horseriding_join = customers_zip.join(delivery_zip, "customer_id").join(sports, "customer_id").join(customer_orders, "customer_id").join(orders, "order_id")
horseriding_join.show(5)

+--------+-----------+--------+-------+----------+-----+
|order_id|customer_id|zip_cust|zip_del|     sport|value|
+--------+-----------+--------+-------+----------+-----+
|10187796|   83072138|   27535|  27535|  p3ywanie|   55|
|10187796|   83072138|   27535|  98096|  p3ywanie|   55|
|10187886|   24956203|   43374|  43374|  p3ywanie|  779|
|10187886|   24956203|   43374|  34248|  p3ywanie|  779|
|10187943|   45765186|   73510|  73510|nurkowanie|  676|
+--------+-----------+--------+-------+----------+-----+
only showing top 5 rows



In [35]:
horse_riding_filtered = horseriding_join.filter(horseriding_join.sport == "jeYdziectwo")
horse_riding_filtered.show(5)

+--------+-----------+--------+-------+-----------+-----+
|order_id|customer_id|zip_cust|zip_del|      sport|value|
+--------+-----------+--------+-------+-----------+-----+
|10188175|   59297032|   98124|  97987|jeYdziectwo|  249|
|10188175|   59297032|   98124|  98124|jeYdziectwo|  249|
|10190729|   14982223|   10538|  17286|jeYdziectwo|  150|
|10190729|   14982223|   10538|  10538|jeYdziectwo|  150|
|10191383|   30027060|   10297|  73186|jeYdziectwo|  511|
+--------+-----------+--------+-------+-----------+-----+
only showing top 5 rows



In [36]:
avg_order_value_df = horse_riding_filtered.groupBy("customer_id").agg(avg("value").alias("avg_order_value"))

In [37]:
overall_avg_order_value = avg_order_value_df.agg(avg("avg_order_value"))
overall_avg_order_value.show()

23/02/28 13:02:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/28 13:02:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/28 13:02:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/28 13:02:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+--------------------+
|avg(avg_order_value)|
+--------------------+
|  25928.559230661373|
+--------------------+



Srednia wartosc zamowienia dla jezdziectwa wyniosla **25928** lecz podobnie jak w przypadku sredniej ceny zamowienia dla wszystkich ze sportow mediana moze byc lepszym wskaznikiem 

# 

# 